In [2]:
pip install pandas konlpy tqdm 

  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 6.0 MB/s  0:00:01 eta 0:00:01
Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.4/582.4 kB 4.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 6.1 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 4.9 MB/s  0:00:01m0:00:0100:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [konlpy]2m6/8 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install wordcloud matplotlib

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import re
import gc
import time
import pandas as pd
from konlpy.tag import Okt
from tqdm import tqdm

In [17]:
# 설정 (경로 수정)
INPUT_CSV = "csv/melon_genre_steady_songs_with_lyrics.csv"
OUT_CSV   = "csv/melon_song_words.csv"
BATCH_SLEEP = 0.05   # 각 곡 처리 후 짧게 쉬기
SAVE_EVERY = 50      # 이 값만큼 곡 처리할 때마다 파일에 한번 더 flush(안전성)

EN_STOP = {"the","and","a","to","of","in","on","for","you","i","me","my","is","it","we","be","that","this"}
KO_STOP = {"나","너","우리","그대","너희","이","그","저","그것","이것","저것","너의","하다"}
okt = Okt()

In [6]:
pip install wordcloud matplotlib


Defaulting to user installation because normal site-packages is not writeable
  Using cached wordcloud-1.9.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (3.4 kB)
Using cached wordcloud-1.9.4-cp39-cp39-macosx_11_0_arm64.whl (168 kB)
Note: you may need to restart the kernel to use updated packages.


In [25]:
from collections import Counter

def extract_words_light(lyrics: str):
    """
    한 곡 처리:
     - 한국어: 명사/동사/형용사만 추출, 한 글자 제거, KO_STOP 제거
     - 영어: 정규표현식 토큰화, 소문자화, EN_STOP 제거
    """
    if not isinstance(lyrics, str) or not lyrics.strip():
        return []

    text = re.sub(r"[^가-힣A-Za-z\s]", " ", lyrics)
    words = []

    # 한국어
    ko_text = " ".join(re.findall(r"[가-힣]+", text))
    if ko_text:
        try:
            tokens = okt.pos(ko_text, stem=True)
            for w, pos in tokens:
                w = w.replace("\u200b", "").strip()

                if pos in ("Noun","Verb","Adjective") and len(w) > 1 and w not in KO_STOP:
                    words.append(w.strip())   # ✅ (변경됨)
        except Exception:
            pass

    # 영어
    en_text = " ".join(re.findall(r"[A-Za-z]+", text))
    if en_text:
        en_tokens = re.findall(r"[A-Za-z]{2,}", en_text)
        for t in en_tokens:
            t = t.lower().strip()
            if t not in EN_STOP:
                words.append(t)

    return words


# -----------------------
# CSV 읽고 전체 단어 모으기
# -----------------------
df = pd.read_csv(INPUT_CSV)
genres = df['genre'].unique()
for i, genre in enumerate(genres, 1):   # i 자동 증가
    group = df[df['genre'] == genre]
    all_words = []
    for idx, row in tqdm(group.iterrows(), total=len(group), desc=f"Processing {genre}"):
        lyrics = row.get("lyrics", "")
        words = extract_words_light(lyrics)
        all_words.extend(words)

print(f"[INFO] 전체 {len(all_words)}개의 단어 추출 완료")


# -----------------------
# 워드클라우드 생성
# -----------------------
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 🔄 변경: 장르별로 그룹을 돌면서 워드클라우드 생성 및 저장
for genre, group in df.groupby("genre"):   # ✅ genre 기준으로 그룹핑
    all_words = []
    for idx, row in group.iterrows():
        lyrics = row.get("lyrics", "")
        words = extract_words_light(lyrics)
        all_words.extend(words)

    if not all_words:
        print(f"[WARN] 장르 '{genre}' 단어 없음 → 스킵")
        continue

    # ✅ Counter로 단어 빈도 집계
    word_freq = Counter(all_words)

    # ✅ 빈도 기반 워드클라우드 생성
    wc = WordCloud(
        font_path=font_path,
        width=800, height=400,
        background_color="white"
    ).generate_from_frequencies(word_freq)

    # ✅ 장르별 파일로 저장
    safe_genre = re.sub(r'[\/:*?"<>|]', "_", str(genre))
    save_path = f"wordcloud_{safe_genre}.png"
    wc.to_file(save_path)

    print(f"✅ '{genre}' 장르 워드클라우드 저장 완료 → {save_path}")

Processing 록/메탈: 100%|████████████████████| 200/200 [00:00<00:00, 240.10it/s]


[INFO] 전체 18872개의 단어 추출 완료
✅ '댄스' 장르 워드클라우드 저장 완료 → wordcloud_댄스.png
✅ '랩/힙합' 장르 워드클라우드 저장 완료 → wordcloud_랩_힙합.png
✅ '록/메탈' 장르 워드클라우드 저장 완료 → wordcloud_록_메탈.png
✅ '발라드' 장르 워드클라우드 저장 완료 → wordcloud_발라드.png
